#Summary Statistics for the Smith Dictionaries for Biography and Geography
##(the dictionary of daily life TBD)



In [9]:
import re
import pandas as pd

curvolume = ''
countables = {}
def datesummary(curdict,moddates,s):
    global curvolume
    
    matchs = '<date when-custom="(\-[0-9\-]+|[0-9\-]+)[^>]*>[^<]+'
    while(re.search(matchs,s)):
        

        m = re.search(matchs,s)
        curdate = m[1]
        if(curdate in curdict):
            curdict[curdate] = curdict[curdate] + 1
        else:
            curdict[curdate] =  1            
        s = re.sub(matchs,' ',s,1)
        
    #now look for untagged dates -- we assume that four digit numbers starting with 1 are dates    
    
    s = re.sub('<ref[^>]+>[^<]+</ref>',' ',s)

    s = re.sub('<bibl[^>]*>[^<]+</bibl>',' ',s)
    s = re.sub('<date[^>]*>[^<]+</date>',' ',s)
    s = re.sub('<measure[^>]+>[^<]+</measure>',' ',s)
    s = re.sub('<figure[^>]+>',' ',s)
    s = re.sub('<pb[^>]+>',' ',s)

    m = re.search('<milestone unit="volume" n="([123])"/>',s)
    if(m):
        curvolume = m[1]
    matchs = '[\s](1[0-9][0-9][0-9]\\b)'    
    while(re.search(matchs,s)):
            m = re.search(matchs,s)
            curdate = m[1]
            if(re.search('1084',curdate)):
                print(s)
            if(curvolume):
                curdate = curdate + '-' + curvolume
            if(curdate in moddates):
                moddates[curdate] = moddates[curdate] + 1 
            else:
                moddates[curdate] = 1
            s = re.sub(matchs,' ',s,1)

def numbernouns(s):
    units = '([A-Z][a-z]+\s+talents|geog\.\s+[Mm]iles|smaller\s+vessels|[A-Z][a-z]+\s+citizens|[A-Z][a-z]+\s+mercenaries|M\.\s*P\.|[A-Z][a-z]+\s+[Aa]cres|[A-Z][a-z]+\s+[mM]iles|[A-Z][a-z]+\s+[Ff]eet|[A-Z][a-z]+\s+yards|sq\.\s+miles|square\s+miles|armed\s+[a-zA-Z]+|heav[ily]-armed\s+[A-Za-z]+|[A-Za-z\-]+)'
    matchs = '([2-9][0-9][0-9][0-9]|[0-9]+,[0-9][0-9,]+[0-9])\s+' + units
    
    s = re.sub('<date[^>]+>[^<]+</date>',' ',s)
    s = re.sub('<bibl[^>]+>[^<]+</bibl>',' ',s)
    s = re.sub('<measure[^>]+>[^<]+</measure>',' ',s)
    while(re.search(matchs,s)):
        m = re.search(matchs,s)
        curnumber = m[1]
        curnoun = m[2]
        if(curnoun in countables):
            countables[curnoun] = countables[curnoun] + 1
        else:
            countables[curnoun] = 1
        
        print(curnumber,curnoun)
    
        s = re.sub(matchs,'',s,1)
    
    
def entrysummary(curdict,curentryid,s):
    
    m = re.search('entry"\s+xml:id="([^"]+)"',s)
    if(m):
        curentryid = m[1]
        curdict[curentryid] = 0
        return(curentryid)
    
    if(curentryid):
        curdict[curentryid] = curdict[curentryid] + len(re.sub('<[^>]+>','',s).split())
    return(curentryid)
    
    
def assignauth(curentryid,authtab,s):
    m = re.search('target="author\.([^"]+)"',s)
    if(m):
        if(not m[1] in authtab):
            authtab[m[1]] = []
        authtab[m[1]].append(curentryid)
    
def getsmithstats(fname):
    datedict = {}
    moddatedict = {}
    entrydict = {}
    authors = {}
    f = open(fname)
    curentryid = ''
    for l in f:
        datesummary(datedict,moddatedict,l)
        curentryid = entrysummary(entrydict,curentryid,l)
        assignauth(curentryid,authors,l)
        #numbernouns(l)
    
    f.close()
    return(datedict,moddatedict,entrydict,authors)

geodates,geomoddates,geoentries,geoauthors = getsmithstats('002/smithgeo-work.xml')
avggeo = sum(geoentries.values())/len(geoentries)

curvolume = ''
biodates,biomoddates,bioentries,bioauthors = getsmithstats('003/smithbio-work.xml')
avgbio = sum(bioentries.values())/len(bioentries)

avggeodates = sum(geodates.values())/len(geoentries)
avgbiodates = sum(biodates.values())/len(bioentries)

summstats = []
summstats.append(['Geo',len(geoentries),sum(geoentries.values()),avggeo,len(geodates),sum(geodates.values()),avggeodates])
summstats.append(['Bio',len(bioentries),sum(bioentries.values()),avgbio,len(biodates),sum(biodates.values()),avgbiodates])
df_summary = pd.DataFrame(summstats,columns = ['work','entries','total words','avg words','Unique Dates','Total Dates','Dates per entry'])
i = 0

bioentlist = sorted(bioentries,key=bioentries.get,reverse=True)
geoentlist = sorted(geoentries,key=geoentries.get,reverse=True)


geodatelist = sorted(geodates,key=geodates.get,reverse=True)
biodatelist = sorted(biodates,key=biodates.get,reverse=True)

data = []
datedata = []
for i in range(0,100):
    #print(bioentlist[i],bioentries[bioentlist[i]],geoentlist[i],geoentries[geoentlist[i]],sep='\t')
    data.append([bioentlist[i],bioentries[bioentlist[i]],geoentlist[i],geoentries[geoentlist[i]]])
    datedata.append([biodatelist[i],biodates[biodatelist[i]],geodatelist[i],geodates[geodatelist[i]]])
#df_entries = pd.DataFrame(data, columns = ['BioEntry','Entrysize' 'GeoEntry','Entrysize'])

    
df_entries = pd.DataFrame(data, columns = ['BioEntry','Entrysize' ,'GeoEntry','Entrysize'])
df_dates = pd.DataFrame(datedata, columns = ['BioDates','DateCnt' ,'GeoDates','DateCnt'])

print('Stats for the Smith Biographical and Geographical Dictionaries')

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 2,
                       ):
    display(df_summary)
    
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 2,
                       ):
    display(df_entries[0:30])
    
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 2,
                       ):
    display(df_dates[0:100])



<s n="r01"/>They were repulsed by the vigilance and energy of pope Leo IV., who repaired the ancient walls, restored fifteen towers which had been overthrown, and enclosed the quarter of the Vatican; on which in  </date> he bestowed his blessing and the title of <placeName type="modern" rend="ital">Città Leonina</placeName>, or Leonine city (now the <title>Borgo di S. Pietro</title>). (Anastasius, <title>V. Leon. IV.</title>) In the period between  and 1084 Rome was thrice fruitlessly besieged by the emperor Henry IV., who, however, by means of corruption at last succeeded in gaining possession of it; but the ruins of the Septizonium, defended by the nephew of Pope Gregory VII., resisted all the attacks of Henry's forces.

Stats for the Smith Biographical and Geographical Dictionaries


,work,entries,total words,avg words,Unique Dates,Total Dates,Dates per entry
0,Geo,11594,2770489,238.96,1444,5890,0.51
1,Bio,19900,4077827,204.92,2275,22188,1.11


,BioEntry,Entrysize,GeoEntry,Entrysize
0,cicero-bio-7,42106,roma-geo,144831
1,aristoteles-bio-2,29441,athenae-geo03,53159
2,caesar-bio-19,18976,gallia-transalpina-geo,28541
3,justinianus-i-flavius-anicius-bio-1,16357,italia-geo,26234
4,plato-bio-2,14753,carthago-geo,26005
5,homerus-bio-1,14476,palaestina-geo,20020
6,pheidias-bio-1,13729,etruria-geo,18349
7,pompeius-bio-22,13255,thracia-geo,17033
8,ptolemaeus-claudius-bio-1,12851,hispania-geo,16619
9,eucleides-bio-1,12172,phoenicia-geo,16578


,BioDates,DateCnt,GeoDates,DateCnt
0,-43,173,-218,44
1,-44,153,-340,44
2,-49,153,-52,43
3,-45,125,-480,40
4,-54,124,-191,39
5,-46,121,-200,37
6,-63,118,-58,33
7,001-100,115,-396,32
8,-48,113,-49,32
9,69,108,-338,31


In [10]:
newdict = (sorted(biomoddates.keys(),reverse=True))
for i in range(0,len(newdict)):
    print(newdict[i],biomoddates[newdict[i]])

1848-3 15
1847-3 10
1846-3 27
1846-2 4
1846-1 1
1845-3 14
1845-2 14
1844-3 22
1844-2 16
1844-1 1
1843-3 28
1843-2 21
1843-1 14
1842-3 24
1842-2 14
1842-1 20
1841-3 23
1841-2 23
1841-1 24
1840-3 19
1840-2 30
1840-1 22
1839-3 18
1839-2 19
1839-1 13
1838-3 22
1838-2 19
1838-1 21
1837-3 14
1837-2 24
1837-1 19
1836-3 24
1836-2 19
1836-1 28
1835-3 24
1835-2 17
1835-1 21
1834-3 26
1834-2 9
1834-1 13
1833-3 21
1833-2 16
1833-1 10
1832-3 16
1832-2 21
1832-1 20
1831-3 27
1831-2 19
1831-1 17
1830-3 21
1830-2 22
1830-1 19
1829-3 28
1829-2 11
1829-1 26
1828-3 25
1828-2 29
1828-1 27
1827-3 33
1827-2 21
1827-1 26
1826-3 23
1826-2 13
1826-1 26
1825-3 16
1825-2 20
1825-1 18
1824-3 19
1824-2 13
1824-1 18
1823-3 18
1823-2 16
1823-1 17
1822-3 20
1822-2 10
1822-1 10
1821-3 9
1821-2 10
1821-1 17
1820-3 13
1820-2 11
1820-1 12
1819-3 12
1819-2 11
1819-1 14
1818-3 8
1818-2 9
1818-1 8
1817-3 9
1817-2 9
1817-1 12
1816-3 14
1816-2 10
1816-1 15
1815-3 14
1815-2 10
1815-1 13
1814-3 8
1814-2 9
1814-1 8
1813-3 10
181

In [99]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df_entries[1:30])

,BioEntry,Entrysize,GeoEntry,Entrysize
1,aristoteles-bio-2,29438,athenae-geo03,53157
2,caesar-bio-19,18976,gallia-transalpina-geo,28541
3,justinianus-i-flavius-anicius-bio-1,16357,italia-geo,26234
4,plato-bio-2,14744,carthago-geo,26003
5,homerus-bio-1,14476,palaestina-geo,20020
6,pheidias-bio-1,13727,etruria-geo,18347
7,pompeius-bio-22,13255,thracia-geo,17033
8,ptolemaeus-claudius-bio-1,12849,hispania-geo,16619
9,eucleides-bio-1,12172,phoenicia-geo,16577
10,galenus-claudius-bio-1,11159,gallia-cisalpina-geo,15860


In [47]:
df_entries = pd.DataFrame(data, columns = ['BioEntry','Entrysize' ,'GeoEntry','Entrysize'])

In [47]:
df_entries = pd.DataFrame(data, columns = ['BioEntry','Entrysize' ,'GeoEntry','Entrysize'])

In [48]:
df_entries

,BioEntry,Entrysize,GeoEntry,Entrysize
0,cicero-bio-7,42104,roma-geo,144816
1,aristoteles-bio-2,29438,athenae-geo03,53157
2,caesar-bio-19,18976,gallia-transalpina-geo,28541
3,justinianus-i-flavius-anicius-bio-1,16357,italia-geo,26234
4,plato-bio-2,14744,carthago-geo,26003
5,homerus-bio-1,14476,palaestina-geo,20020
6,pheidias-bio-1,13727,etruria-geo,18347
7,pompeius-bio-22,13255,thracia-geo,17033
8,ptolemaeus-claudius-bio-1,12849,hispania-geo,16619
9,eucleides-bio-1,12172,phoenicia-geo,16577


In [54]:
arts = 0
for foo in geoauthors:
    arts = arts + len(geoauthors[foo])
    print(foo,len(geoauthors[foo]))
print(arts)

E.H.B 1013
W.B.D 207
P.S 497
H.W 4
W.R 8
L.S 1059
G.L 1151
G.W 328
R.G.L 194
E.B.J 709
J.S.H 12
V 559
G.F.B 1
C.R.S 19
T.H.D 802
G.B 1
J.R 54
6618


In [151]:
for foo in sorted(countables,key=countables.get,reverse=True):
    print(foo,countables[foo])

men 303
feet 239
stadia 158
foot 87
inhabitants 86
of 81
horse 67
persons 34
sesterces 33
to 27
years 27
souls 26
talents 25
were 24
in 21
or 20
pieces 19
asses 17
cavalry 16
and 16
citizens 15
prisoners 13
infantry 13
Romans 13
English feet 12
colonists 12
paces 12
drachmae 11
hoplites 10
spectators 10
pounds 10
square miles 9
mercenaries 9
soldiers 9
Greeks 9
slaves 8
for 8
volumes 8
medimni 8
veterans 8
yards 7
verses 7
miles 6
Paris feet 6
lines 6
M. P. 5
troops 5
taken 5
fighting 5
denarii 5
Greek mercenaries 5
armed men 4
horsemen 4
more 4
sq. miles 4
Roman 4
at 4
Macedonians 4
escaped 4
hexameter 4
B 3
new 3
mares 3
M.P. 3
Athenian citizens 3
ft 3
as 3
square 3
from 3
families 3
war-chariots 3
horses 3
settlers 3
m 3
Locrians 3
Jews 3
lots 3
light-armed 3
Crotoniats 3
Samnites 3
drachmas 3
are 3
Macedonian 3
enemies 3
Goths 3
Gauls 2
houses 2
geog. miles 2
free 2
bushels 2
Roman citizens 2
heavy-armed infantry 2
Athenian 2
Galli 2
slain 2
had 2
being 2
auxiliaries 2
on 2
passus 

In [199]:
l = ' n="1066"'

matchs = '[\\s](1[0-9][0-9][0-9]\\b)'

m = re.search(matchs,l)
if(m):
    print('hit')

In [77]:
avg

'204.9'

In [14]:
l = 'bout 1953 l. 2 s.  6d.'

matchs = '([0-9]+,[0-9][0-9,]+[0-9]|[0-9]+)\s*l\.\s*([0-9]+)\s*s\.\s+([0-9]+)\s*d\.'
while(re.search(matchs,l)):
        m = re.search(matchs,l)
        pounds = re.sub(',','',m[1])
        l = re.sub(matchs,'<measure quantity="'+pounds+'.\g<2>.\g<3>">\g<1>l.</z> \g<2>s. \g<3>d.</measure>',l)
 


In [33]:
import re

j = 0



f = open('002/smithgeo-work.xml')
#f = open('003/smithbio-work.xml')

lastdate = ''
lastbib = ''
i = 0
j = 0

notyears = ['Argonautica','Dicaearchus','Editions','Epigram','Eutrop','Gennadius','Hephaestion','Liberalis','Lycophron','Petronius','Theogony','appended','chapter','epigrams','wrote']
notyears.append('Apollo')
notyears.append('Aphrodite')
notyears.append('Aristophanes')
notyears.append('Artemis')
notyears.append('Avienus')
notyears.append('Chius')
notyears.append('Demeter')
notyears.append('Festus')
notyears.append('Jornandes')
notyears.append('Obsequens')
notyears.append('Orelli')
notyears.append('Orpheus')
notyears.append('Periegetes')
notyears.append('Phrygius')
notyears.append('Scymnus')
notyears.append('Thesmophoriazusae')

for l in f:
    
    m = re.search('<bibl n="([A-Z][a-z]+)',l)
    if(m):
        if(not m[1] in lastbib):
            lastbib.append( m[1])
        
    m = re.search('<date when-custom="([^"]+)"',l)
    if(m):
            lastdate = m[1]
            

    
    matchs = '\\b([Ii]n\s+)([0-9]+\.[0-9]+\.[0-9]+|[0-9]+\.[0-9]+)'

    l = re.sub(matchs,'\g<1><ref>\g<2></ref>',l)
    

    
    m = re.search('entry" xml:id="([^"]+)',l)
    if(m):
        curentryid = m[1]
        lastdate = 'nolast'
        lastbib = []
    
    matchs = '\\b([a-zA-Z]+)\s+([a-zA-Z]+)\s+\(([0-9]+)\)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)
    
        if(m and int(m[3]) > 9 and int(m[3]) < 1000 and not m[2] in notyears):
            j = j + 1
            if(re.search('^-',lastdate) or 'Liv' in lastbib or 'Cic' in lastbib or 'Thuc' in lastbib or 'Dionys' in lastbib):
                newdate = '-' + m[3]
            else:
                newdate = m[3]
            print(j,'paren',curentryid,lastdate,'Cic' in lastbib,lastbib,m[1],m[2],m[3],newdate)
            l = re.sub(matchs,'\g<1> \g<2> (<date when-custom="'+newdate+'">\g<3></date>)',l,1)
        else:
            l = re.sub(matchs,'\g<1> \g<2> (<ref>\g<3></ref>)',l,1)
    
    matchs = '\\b([Ii]n)\s([0-9]+)'
    while(re.search(matchs,l)):
        m = re.search(matchs,l)            
        newdate = m[2]
        if(re.search('^-',lastdate) or  'Plb' in lastbib or 'Liv' in lastbib or 'Cic' in lastbib):
            if(int(m[2])< 900):
                newdate = '-' + m[2]
            else:
                newdate = m[2]
        else:
            newdate = m[2]
        if(len(newdate) == 1):
            shows = l
        else:
            shows = ''
        i = i + 1
        if(lastdate == 'nolast' and not lastbib == 'nobib' and int(newdate)  < 1000):
            print(i,curentryid,'newdate',lastdate,lastbib,newdate,shows)
        l =re.sub(matchs,'\g<1> <date when-custom="'+newdate+'">\g<2></date>',l,1)

print('done')
f.close()

1 paren aegina-geo -480 True ['Paus', 'Ov', 'Hdt', 'Ath', 'Thuc', 'Cic'] of Aegospotami 404 -404
2 paren agraei-geo -426 False ['Thuc'] years afterwards 424 -424
3 paren alani-geo nolast False ['Luc', 'Plin'] he says 308 308
4 paren alani-geo 379 False ['Luc', 'Plin', 'Ptol'] conquered Spain 418 418
5 paren alani-geo 379 False ['Luc', 'Plin', 'Ptol'] their defection 451 451
6 paren ambracia-geo -432 False ['Hdt', 'Plb', 'Scymnus'] years afterwards 430 -430
7 paren ambracia-geo -432 False ['Hdt', 'Plb', 'Scymnus'] following year 429 -429
8 paren amphipolis-geo -360 False ['Thuc', 'Hdt'] his accession 359 -359
9 paren amphipolis-geo -360 False ['Thuc', 'Hdt'] following year 358 -358
10 paren arcadia-geo02 -365 False ['Paus', 'Plin', 'Apollon', 'Hdt', 'Ath', 'Hom', 'Diod', 'Thuc', 'Xen', 'Plut'] Olympic games 364 -364
11 paren argos-amphilochicum-geo -432 False ['Thuc'] years afterwards 430 -430
12 paren atella-geo -211 False ['Stephanus', 'Liv'] next year 210 -210
13 paren jerusalem-geo 

In [10]:
'epigrams' in notyears

True